In [15]:
import gdown

# The extracted Google Drive file ID
file_id = '1mah8we6k2Ts86CI5phjSEhpcm4XfbRux'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'CitrusV2.zip'  # Choose a name for your file

# Download the file
gdown.download(url, output, quiet=False)

print(f"File has been successfully downloaded to: {output}")


Downloading...
From (original): https://drive.google.com/uc?id=1mah8we6k2Ts86CI5phjSEhpcm4XfbRux
From (redirected): https://drive.google.com/uc?id=1mah8we6k2Ts86CI5phjSEhpcm4XfbRux&confirm=t&uuid=e3ac74a1-d025-4ebf-a55b-65f9f44fbb1d
To: /workspace/CitrusV2.zip
100%|██████████| 40.6M/40.6M [00:00<00:00, 66.4MB/s]


File has been successfully downloaded to: CitrusV2.zip


In [16]:
import zipfile as zf
files = zf.ZipFile("CitrusV2.zip", 'r')
files.extractall('Citrus')
files.close()

In [12]:
os.listdir('Citrus/CitrusV2')

FileNotFoundError: [Errno 2] No such file or directory: 'Citrus/CitrusV2'

In [19]:
import os

# Specify the directory path
directory_path = '../workspace/Citrus/CitrusV1'

# List all files in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    try:
        # Check if it is a file
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"{file_path} has been deleted.")
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")


In [4]:
import tensorflow as tf
import imageio
import numpy as np
import os

def read_images(dir):

    supported_extensions = [".jpg", ".jpeg", ".png"]
    image_list = []
    count = 0
    # Walk through the directory and read images
    for root, _, files in os.walk(dir):
        for file in files:
            file_extension = os.path.splitext(file)[-1].lower()

            # Check if the file is a .jpg or .jpeg image
            if file_extension in supported_extensions:
                image_path = os.path.join(root, file)
                try:
                    image = imageio.imread(image_path)
                    image1 = image

                    image = np.asarray(image)
                    del image1
                    image_list.append(image)
                except Exception as e:
                    print(f"Error reading image {image_path}: {e}")

            count += 1

            if count % 100 == 0:
              print(str(count) + " images read")


    return image_list



2024-03-24 09:49:17.235163: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 09:49:17.235230: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 09:49:17.235283: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
import imgaug as ia
import imgaug.augmenters as iaa
import random
import copy

def add_gaussian_noise(images, mean_range=(0, 15), std_range=(0, 0.15)):
    ia.seed(1)
    # Define the augmentation pipeline
    seq = iaa.Sequential([
        iaa.AdditiveGaussianNoise(loc=mean_range, scale=(0, 0.2*255))
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    # images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images]

    return augmented_images

def random_crop(images, crop_percent=(0.1, 0.4)):
    ia.seed(1)
    # Define the augmentation pipeline
    seq = iaa.Sequential([
        iaa.Crop(percent=crop_percent)
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    # images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images]

    return augmented_images

def random_rotate(images, rotation_range=(-360, 360)):
    ia.seed(1)
    # Define the augmentation pipeline
    seq = iaa.Sequential([
        iaa.Rotate(rotate=rotation_range)
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    # images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images]

    return augmented_images

def invert_images(images):
    ia.seed(1)
    # Define the augmentation pipeline
    seq = iaa.Sequential([
        iaa.Add(value=(-20, 20))
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    # images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images]

    return augmented_images


def adjust_brightness(images, brightness_range=(-65, 65)):
    ia.seed(1)
    # Define the augmentation pipeline for adjusting brightness
    seq = iaa.Sequential([
        iaa.Multiply((1.0 + brightness_range[0] / 100.0, 1.0 + brightness_range[1] / 100.0))
    ])


    augmented_images = [seq(image=image) for image in images]

    return augmented_images


def scale_images(images, scale_factor = (0.3, 1.8)):
    ia.seed(1)
    # Define the augmentation pipeline for scaling images
    seq = iaa.Sequential([
        iaa.Affine(scale=scale_factor)
    ])

    augmented_images = [seq(image=image) for image in images]

    return augmented_images

def add_contrast(images, contrast_factor=(0.5, 1.5)):
    ia.seed(1)
    # Define the augmentation pipeline for adding contrast
    seq = iaa.Sequential([
        iaa.ContrastNormalization(alpha=contrast_factor)
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images_np]

    return augmented_images

def flip_images(images, flip_probability=0.5):
    ia.seed(1)
    # Define the augmentation pipeline for randomly flipping images
    seq = iaa.Sequential([
        iaa.Sometimes(flip_probability, iaa.Fliplr(1.0)),  # Horizontal flips
        iaa.Sometimes(flip_probability, iaa.Flipud(1.0))   # Vertical flips
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images_np]

    return augmented_images


# discard images according to ratio
def discard_images(images, discard_ratio=0.5):
    random.seed(10)
    # Calculate the number of images to discard based on the discard_ratio
    num_images_to_discard = int(len(images) * discard_ratio)

    # Create a copy of the input list to avoid modifying the original list
    remaining_images = images[:]

    # Randomly discard a portion of the images
    random.shuffle(remaining_images)
    remaining_images = remaining_images[num_images_to_discard:]

    return remaining_images




def resize_images(images_list, width=128, height=128):
    ia.seed(1)
    # Define the resize augmentation
    resize_augmenter = iaa.Resize({"height": height, "width": width})

    resized_images = []

    for image in images_list:
        # Ensure the image is in RGB format (imgaug uses RGB by default)
        if image.shape[-1] == 1:  # Grayscale image with single channel
            image = np.repeat(image, 3, axis=-1)

        # Apply the resize augmentation
        augmented_image = resize_augmenter.augment_image(image)

        # Append the augmented image to the result list
        resized_images.append(augmented_image)

    del images_list[:]
    return resized_images

def keep_n_images(images, n_to_keep):
    random.seed(10)
    if n_to_keep >= len(images):
        return images  # Keep all images if n_to_keep is greater than or equal to the image count

    # Randomly shuffle the images list
    random.shuffle(images)

    # Keep the first n_to_keep images and discard the rest
    kept_images = images[:n_to_keep]

    # Create a copy of the kept images list
    kept_images_copy = copy.deepcopy(kept_images)

    # Clear the original images list to free memory
    del images[:]

    return kept_images_copy
import cv2
def normalize_images(image_list):

  for i in range(len(image_list)):
      image = image_list[i].astype(np.float32) / 255.0
      image_list[i] = image

def discard_images(lst, percent_to_discard):
    # Calculate the number of images to discard
    num_images_to_discard = int(len(lst) * percent_to_discard)
    
    # Generate a list of unique indices to discard
    indices_to_discard = random.sample(range(len(lst)), num_images_to_discard)
    
    # For each index to discard, if there's a need for explicit clean-up (e.g., closing a file or freeing resources), do it here
    for index in indices_to_discard:
        image = lst[index]
        # If the image is an object that requires closing or special disposal, do it here
        # For example, if image objects are from PIL, you might need to call image.close() if they are open files
        # Note: This is only necessary if your image objects require explicit clean-up
    
    # Create a new list excluding the indices to discard
    modified_lst = [lst[i] for i in range(len(lst)) if i not in indices_to_discard]
    
    return modified_lst


In [6]:
base_train_dir = '../workspace/Citrus/CitrusV1/Train/'
base_test_dir = '../workspace/Citrus/CitrusV1/Val/'

# Train data directories
black_spot_train = read_images(os.path.join(base_train_dir, 'Black spot'))
canker_train = read_images(os.path.join(base_train_dir, 'Canker'))
greening_train = read_images(os.path.join(base_train_dir, 'Greening'))
healthy_train = read_images(os.path.join(base_train_dir, 'Healthy'))  # Included explicitly for the 'Healthy' class
melanose_train = read_images(os.path.join(base_train_dir, 'Melanose'))

# Test data directories
black_spot_test = read_images(os.path.join(base_test_dir, 'Black spot'))
canker_test = read_images(os.path.join(base_test_dir, 'Canker'))
greening_test = read_images(os.path.join(base_test_dir, 'Greening'))
healthy_test = read_images(os.path.join(base_test_dir, 'Healthy'))  # Included explicitly for the 'Healthy' class
melanose_test = read_images(os.path.join(base_test_dir, 'Melanose'))

# For training classes
print("Length of Black spot train:", len(black_spot_train))
print("Length of Canker train:", len(canker_train))
print("Length of Greening train:", len(greening_train))
print("Length of Healthy train:", len(healthy_train))
print("Length of Melanose train:", len(melanose_train))

# For test classes
print("Length of Black spot test:", len(black_spot_test))
print("Length of Canker test:", len(canker_test))
print("Length of Greening test:", len(greening_test))
print("Length of Healthy test:", len(healthy_test))
print("Length of Melanose test:", len(melanose_test))

# Calculate the lengths of all train parts
train_lengths = len(black_spot_train) + len(canker_train) + len(greening_train) + \
                len(healthy_train) + len(melanose_train)

# Calculate the lengths of all test parts
test_lengths = len(black_spot_test) + len(canker_test) + len(greening_test) + \
               len(healthy_test) + len(melanose_test)

# Print the results
print("Total length of all train parts:", train_lengths)
print("Total length of all test parts:", test_lengths)

# Calculate the total length of all parts
total_length = train_lengths + test_lengths

# Print the results
print("Total length of all parts:", total_length)


/tmp/ipykernel_13428/1632636461.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)


100 images read
100 images read
100 images read
Length of Black spot train: 118
Length of Canker train: 113
Length of Greening train: 142
Length of Healthy train: 40
Length of Melanose train: 8
Length of Black spot test: 17
Length of Canker test: 17
Length of Greening test: 21
Length of Healthy test: 6
Length of Melanose test: 2
Total length of all train parts: 421
Total length of all test parts: 63
Total length of all parts: 484


In [7]:
import os
os.listdir('Citrus/CitrusV1/Train')

['Black spot', 'Canker', 'Greening', 'Healthy', 'Melanose']

In [8]:
import os
import numpy as np

# Assuming base_train_dir and base_test_dir variables are already defined
base_train_dir = 'Tomato/Train'
base_test_dir = 'Tomato/Val'

# Function to process and augment images for a given class
def process_images(class_name, base_dir):
    images = read_images(os.path.join(base_dir, class_name))
    images_all = (
        add_gaussian_noise(images) +
        random_crop(images) +
        invert_images(images) +
        adjust_brightness(images) +
        scale_images(images) +
        random_rotate(images)
    )
    return images + images_all

# Process training images for new classes
black_spot_train = process_images('Black_spot', base_train_dir)
canker_train = process_images('Canker', base_train_dir)
greening_train = process_images('Greening', base_train_dir)
healthy_train = process_images('Healthy', base_train_dir)  # 'Healthy' class is common
melanose_train = process_images('Melanose', base_train_dir)

# Process testing images for new classes
black_spot_test = process_images('Black_spot', base_test_dir)
canker_test = process_images('Canker', base_test_dir)
greening_test = process_images('Greening', base_test_dir)
healthy_test = process_images('Healthy', base_test_dir)  # 'Healthy' class is common
melanose_test = process_images('Melanose', base_test_dir)

# Generate training labels
labels_black_spot_train = np.zeros(len(black_spot_train))
labels_canker_train = np.ones(len(canker_train))
labels_greening_train = np.full(len(greening_train), 2)
labels_healthy_train = np.full(len(healthy_train), 3)
labels_melanose_train = np.full(len(melanose_train), 4)

# Combine training labels
labels_train = np.concatenate([
    labels_black_spot_train,
    labels_canker_train,
    labels_greening_train,
    labels_healthy_train,
    labels_melanose_train
])

# Generate testing labels
labels_black_spot_test = np.zeros(len(black_spot_test))
labels_canker_test = np.ones(len(canker_test))
labels_greening_test = np.full(len(greening_test), 2)
labels_healthy_test = np.full(len(healthy_test), 3)
labels_melanose_test = np.full(len(melanose_test), 4)

# Combine testing labels
labels_test = np.concatenate([
    labels_black_spot_test,
    labels_canker_test,
    labels_greening_test,
    labels_healthy_test,
    labels_melanose_test
])

dim = 224  # Desired size for resizing images

# Function to resize images
def resize_images(images, width, height):
    resized_images = [resize_image(image, width, height) for image in images]
    return resized_images

# Resize training images
black_spot_train_resized = resize_images(black_spot_train, dim, dim)
canker_train_resized = resize_images(canker_train, dim, dim)
greening_train_resized = resize_images(greening_train, dim, dim)
healthy_train_resized = resize_images(healthy_train, dim, dim)
melanose_train_resized = resize_images(melanose_train, dim, dim)

# Concatenate resized training images
images_train = np.concatenate([
    black_spot_train_resized,
    canker_train_resized,
    greening_train_resized,
    healthy_train_resized,
    melanose_train_resized
])

# Resize testing images
black_spot_test_resized = resize_images(black_spot_test, dim, dim)
canker_test_resized = resize_images(canker_test, dim, dim)
greening_test_resized = resize_images(greening_test, dim, dim)
healthy_test_resized = resize_images(healthy_test, dim, dim)
melanose_test_resized = resize_images(melanose_test, dim, dim)

# Concatenate resized testing images
images_test = np.concatenate([
    black_spot_test_resized,
    canker_test_resized,
    greening_test_resized,
    healthy_test_resized,
    melanose_test_resized
])


/tmp/ipykernel_13428/1632636461.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)


100 images read
200 images read
300 images read
400 images read
500 images read
600 images read
700 images read
800 images read
900 images read
1000 images read
1100 images read
100 images read


NameError: name 'resize_image' is not defined

In [ ]:
# # # Correcting the discard percentage to match the description
# # discard_percentage = 0.9

# # Overwrite each class variable with the reduced dataset after discarding 52% of images
# # bacterial_spot_all = discard_images(bacterial_spot_all, 1)
# early_blight_all = discard_images(early_blight_all, 0.81)
# healthy_all = discard_images(healthy_all, 0.93) 
# # late_blight_all = discard_images(late_blight_all, 1)
# leaf_mold_all = discard_images(leaf_mold_all, 0.8)
# mosaic_virus_all = discard_images(mosaic_virus_all, 0.16)
# septoria_leaf_spot_all = discard_images(septoria_leaf_spot_all, 0.96)
# spider_mites_all = discard_images(spider_mites_all, 0.96)
# target_spot_all = discard_images(target_spot_all, 0.91)
# # yellow_leaf_curl_virus_all = discard_images(yellow_leaf_curl_virus_all, 1)


In [ ]:


# # Define the percentage of images to discard
# discard_percentage = 0.7  # This means 50% of the data will be discarded

# # Apply the discard_images function to each dataset with the defined discard percentage
# bacterial_spot_train = discard_images(bacterial_spot_train, discard_percentage)
# early_blight_train = discard_images(early_blight_train, discard_percentage)
# healthy_train = discard_images(healthy_train, discard_percentage)
# late_blight_train = discard_images(late_blight_train, discard_percentage)
# leaf_mold_train = discard_images(leaf_mold_train, discard_percentage)
# mosaic_virus_train = discard_images(mosaic_virus_train, discard_percentage)
# septoria_leaf_spot_train = discard_images(septoria_leaf_spot_train, discard_percentage)
# spider_mites_train = discard_images(spider_mites_train, discard_percentage)
# target_spot_train = discard_images(target_spot_train, discard_percentage)
# yellow_leaf_curl_virus_train = discard_images(yellow_leaf_curl_virus_train, discard_percentage)

# # After executing the above lines, each dataset variable will now reference a subset
# that retains only 50% of the original images, since we're





In [ ]:
images_train[0]

In [ ]:

normalize_images(images_train)



In [ ]:
images_train[800]

In [ ]:
normalize_images(images_test)

In [ ]:
#TEST Normalization

In [ ]:
print(len(images_train))
print(len(labels_train))

counts = [0,0,0,0,0]

for label in labels_train:
    counts[int(label)] += 1

counts

In [ ]:

ls1 = images_test
ls2 = images_train

images_test = np.array(images_test)
images_train = np.array(images_train)

del ls1
del ls2


shuffle_indices_train = np.random.permutation(len(images_train))
shuffle_indices_test = np.random.permutation(len(images_test))


images_train = images_train[shuffle_indices_train]
labels_train = labels_train[shuffle_indices_train]

images_test = images_test[shuffle_indices_test]
labels_test = labels_test[shuffle_indices_test]

In [ ]:
# %load custom_callback.py
import tensorflow as tf
import numpy as np
import pandas as pd

# Find accuracy of model
def find_accuracy(test,pred):
    correct = 0
    total = len(test)

    for i in range(len(test)):
        if test[i] == pred[i]:
            correct += 1

    return correct/total


# Map ANN outputs to classes
def get_labels(y_pred_ann):
    labels = []

    for pred in y_pred_ann:
        max_index = 0

        for i in range(len(pred)):
            if pred[i] > pred[max_index]:
                max_index = i

        labels.append(max_index)

    return labels

# This callback prints accuracy by epoch information after each epoch
class Save_Accuracy_By_Epoch(tf.keras.callbacks.Callback):
    def __init__(self, test_data):
        self.X_Test = test_data[0]
        self.Y_Test = test_data[1]
        self.accuracies = []
        self.epochs = []

    def on_epoch_end(self, epoch, logs = None):
        y_pred = self.model.predict(self.X_Test)

        if epoch == 4:
            pass

        y_pred = get_labels(y_pred)
        accuracy = find_accuracy(self.Y_Test, y_pred)
        self.epochs.append(epoch+1)
        self.accuracies.append(accuracy)



        print(self.epochs)
        print(self.accuracies)


# This callback prints metrics for every class after each epoch
class Save_Multiclass_Metrics_By_Epoch(tf.keras.callbacks.Callback):
    def __init__(self, test_data, n_classes, save_after = 10,save_thresh =0.94, save_csv_path = 'results.csv', model_name = 'model.keras', decay_rate = 0.9, min_rate = 0.000032, save_folder_name = "."):
        self.X_Test = test_data[0]
        self.Y_Test = test_data[1]

        self.min_rate = min_rate;
        
        self.epochs = []
        self.n_classes = n_classes
        self.save_after = save_after
        self.save_csv_path = save_csv_path
        self.save_folder_name = save_folder_name
        self.model_name = model_name
        self.max_accuracy = 0
        self.save_thresh = save_thresh

        self.mat_sensitivity = []
        self.mat_specificity = []
        self.mat_precision = []
        self.mat_recall = []
        self.mat_accuracy = []
        self.mat_f1 = []
        self.accuracies = []


        for i in range(n_classes):
            self.mat_sensitivity.append([])
            self.mat_specificity.append([])
            self.mat_precision.append([])
            self.mat_recall.append([])
            self.mat_accuracy.append([])
            self.mat_f1.append([])


    def on_epoch_end(self, epoch, logs = None):
        y_pred = self.model.predict(self.X_Test)
        y_pred = get_labels(y_pred)

        total = len(self.Y_Test)

        correct = 0
        for i in range(len(y_pred)):
            if y_pred[i] == self.Y_Test[i]:
                correct += 1

        accuracy = correct/total
        self.accuracies.append(correct / total)

        best = False

        if accuracy >= self.max_accuracy:
            self.max_accuracy = accuracy
            best = True


        if accuracy > self.save_thresh:
          best = True

        for i in range(self.n_classes):
            TP = 0
            FP = 0
            TN = 0
            FN = 0

            for j in range(len(y_pred)):
                if self.Y_Test[j] == i and y_pred[j] == i:
                    TP += 1
                elif self.Y_Test[j] != i and y_pred[j] == i:
                    FP += 1
                elif self.Y_Test[j] == i and y_pred[j] != i:
                    FN += 1
                elif self.Y_Test[j] != i and y_pred[j] != i:
                    TN += 1

            sensitivity = TP / (TP + FN) if (TP + FN) > 0 else -1
            specificity = TN / (TN + FP) if (TN + FP) > 0 else -1
            precision = TP / (TP + FP) if (TP + FP) > 0 else -1
            recall = TP / (TP + FN) if (TP + FN) > 0 else -1
            accuracy = (TP + TN) / (TP + FN + TN + FP)
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else -1

            self.mat_sensitivity[i].append(sensitivity)
            self.mat_specificity[i].append(specificity)
            self.mat_precision[i].append(precision)
            self.mat_recall[i].append(recall)
            self.mat_accuracy[i].append(accuracy)
            self.mat_f1[i].append(f1)

        self.epochs.append(int(epoch+1))

        print("max Accuracy: ", self.max_accuracy, "Learning Rate: ",self.model.optimizer.learning_rate.numpy())
           

        if self.model.optimizer.learning_rate <= self.min_rate :
           self.model.stop_training = True

        if (epoch + 1) % self.save_after == 0:
            save_to_csv_file(self.save_csv_path, self.mat_sensitivity, self.mat_specificity, self.mat_precision, self.mat_recall, self.mat_accuracy, self.mat_f1, self.accuracies, self.epochs)
            self.model.save(self.model_name)
            if best:
                self.model.save(self.save_folder_name + "/epoch" + str((epoch+1)) + "_" + self.model_name)
            pass
        

 

def save_to_csv_file(path, mat_sensitivity, mat_specificity, mat_precision, mat_recall, mat_accuracy, mat_f1, accuracies, epochs):
    mat_sensitivity = np.transpose(mat_sensitivity)
    mat_specificity = np.transpose(mat_specificity)
    mat_precision = np.transpose(mat_precision)
    mat_recall = np.transpose(mat_recall)
    mat_accuracy = np.transpose(mat_accuracy)
    mat_f1 = np.transpose(mat_f1)
    accuracies = np.reshape(accuracies, (-1,1))
    epochs = np.reshape(epochs, (-1,1))

    mat_join = np.concatenate((epochs,accuracies,mat_sensitivity, mat_specificity, mat_precision, mat_recall, mat_accuracy, mat_f1), axis = 1)

    n = mat_sensitivity.shape[1]


    col_array_sensitivity = list(range(n))
    col_array_specificity = list(range(n))
    col_array_precision = list(range(n))
    col_array_recall = list(range(n))
    col_array_accuracy = list(range(n))
    col_array_f1 = list(range(n))


    for i in range(n):
        col_array_sensitivity[i] = 'sensitivity Class ' + str(col_array_sensitivity[i])
        col_array_specificity[i] = 'specificity Class ' + str(col_array_specificity[i])
        col_array_precision[i] = 'precision Class ' + str(col_array_precision[i])
        col_array_recall[i] = 'recall Class ' + str(col_array_recall[i])
        col_array_accuracy[i] = 'accuracy Class ' + str(col_array_accuracy[i])
        col_array_f1[i] = 'f1 Class ' + str(col_array_f1[i])

    cols = ['Epoch']+['overall_accuracy']+ col_array_sensitivity + col_array_specificity + col_array_precision + col_array_recall + col_array_accuracy + col_array_f1

    mat_join = np.flip(mat_join, axis = 0)



    df = pd.DataFrame(
        columns = cols,
        data  = mat_join
    )

    df.to_csv(path, index= False)


In [ ]:
# One hot encoding
labels_train_encoded = []
for label in labels_train:
    encoding = [0,0,0,0,0]
    encoding[int(label)] = 1
    labels_train_encoded.append(encoding)

labels_train_encoded = np.array(labels_train_encoded)

labels_test_encoded = []
for label in labels_test:
    encoding = [0,0,0,0,0]
    encoding[int(label)] = 1
    labels_test_encoded.append(encoding)

labels_test_encoded = np.array(labels_test_encoded)


In [ ]:
from tensorflow import keras
from keras.layers import Layer
import keras.backend as K
import tensorflow as tf


from keras.datasets import cifar10
from keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten, BatchNormalization, Activation, MaxPooling2D, GlobalMaxPooling2D,\
    Reshape,  multiply, add, Permute

import math
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from keras.regularizers import l2

In [ ]:
K.image_data_format()

In [ ]:
def _conv2d_bn(input,
               filters,
               num_row,
               num_col,
               padding='same',
               strides=(1, 1),
               name=None):

    if name is not None:
        bn_name = '{name}_bn'.format(name=name)
        conv_name = '{name}_conv'.format(name=name)
    else:
        bn_name = None
        conv_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    X = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name) (input)
    X = BatchNormalization(axis=bn_axis, scale=False, name=bn_name) (X)
    X = Activation('relu', name=name)(X)

    return X



In [ ]:

def _tensor_shape(tensor):
    return getattr(tensor, 'shape')

In [ ]:
model = tf.keras.models.Sequential()

raw_model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224,224,3),
    pooling=None
)

model.add(raw_model)

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(5, activation='softmax'))



In [ ]:

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    0.0003,
    decay_steps=5000,
    decay_rate=0.9,
    staircase=True)


opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
loss = tf.keras.losses.CategoricalCrossentropy()

tf.keras.utils.set_random_seed(2)

model.summary()
model.compile(optimizer=opt, loss=loss, metrics = ['acc'])
# model.compile(optimizer='adam', loss=loss, metrics = ['acc'], loss_weights=class_weights)




In [ ]:
history = model.fit(images_train, labels_train_encoded, batch_size=16, epochs=1000, validation_data=(images_test, labels_test_encoded), callbacks=[Save_Multiclass_Metrics_By_Epoch((images_test,labels_test), n_classes=10, save_after=1, save_csv_path="results_model_part1.csv", model_name="model_part1.keras", save_folder_name="Model", min_rate=0.000004)])

In [ ]:
# model = keras.models.load_model('../workspace/Model/epoch99_model_part1.keras')
# model.save('tomato.h5')